In [1]:
%load_ext autoreload
%autoreload 2

import os; import sys; from pathlib import Path
root = Path(os.getcwd()).parents[0]
sys.path.append(str(root))


In [2]:
import joblib
import pandas as pd
from src.model import Regressor
import torch


In [3]:
scaler = joblib.load('../data/scaler.pkl') 
top_X_location = joblib.load('../data/top_X_location.pkl')
ohe = joblib.load('../data/location_one_hot_encoder.pkl')


In [4]:
df = pd.read_csv('../data/booli_1000_pages_raw_20210506.csv')

display(
    df.head(3),
    f'shape = {df.shape}'
)


,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Avgift,Bostadstyp,Driftskostnad,Våning,Byggår,BRF,adress,area_and_n_rooms,location,url,Energiklass
0,5 950 000 kr,4 690 000 kr,+1 260 000 kr (+26.9 %),2021-05-05,108 200 kr/m²,13,3 178 kr/mån,Lägenhet,400 kr/mån,5 tr,1904.0,BRF Bergskyrkan,Bergs­gatan 59,"55 m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4201785,NaN
1,8 000 000 kr,6 695 000 kr,+1 305 000 kr (+19.5 %),2021-05-06,115 900 kr/m²,6,3 876 kr/mån,Lägenhet,599 kr/mån,2 tr,2019.0,BRF Tegeludden 16,Östhammars­gatan 70B,"69 m², 3 rum","Lägenhet, Gärdet",https://www.booli.se/bostad/4099014,NaN
2,6 500 000 kr,4 950 000 kr,+1 550 000 kr (+31.3 %),2021-05-05,97 700 kr/m²,12,5 041 kr/mån,Lägenhet,250 kr/mån,4 tr,1997.0,BRF St.Erik 1,Carl-Gustaf Lindstedts gata 10,"66½ m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4201484,NaN


'shape = (12658, 17)'

In [5]:
def preprocess_inference(df):
    columns_mapping = {
            "Utropspris": "price_ask",
            "Avgift": "montly_fee_cost",
            "Driftskostnad": "montly_operating_cost",
            "Våning": "floor",
            "Byggår": "construction_year",
            "area_and_n_rooms": "area_and_n_rooms",
            "location": "location"
        }

    df = df.rename(
        columns=columns_mapping
    )

    columns_to_keep = columns_mapping.values() # will not be needed 
    df = df[columns_to_keep]

    df['price_ask'] = df['price_ask'].str.strip(' kr').str.replace(' ', '').astype(float)
    df['montly_fee_cost'] = df['montly_fee_cost'].str.strip(' kr/mån').str.replace(' ', '').astype(float)
    df['montly_operating_cost'] = df['montly_operating_cost'].str.strip(' kr/mån').str.replace(' ', '').astype(float)
    df['floor'] = df['floor'].str.strip(' tr').str.replace('½','.5').str.replace('BV','0').astype(float)
    df['construction_year'] = df ['construction_year'].astype(float)

    df[['area', 'n_rooms']] = df['area_and_n_rooms'].str.split(',', expand=True)
    df.drop(columns=['area_and_n_rooms'], inplace=True)
    df['area'] = df['area'].str.strip(' m²').str.replace('½','.5').astype(float)
    df['n_rooms'] = df['n_rooms'].str.strip(' rum').str.replace('½','.5').astype(float)

    df['list_price_per_m2'] = df.price_ask/df.area

    df['location'] = df['location'].str.replace('Lägenhet, ', '').str.lower()
    df.loc[~df['location'].isin(top_X_location), 'location'] = 'other'
    dummies = ohe.transform(df[['location']])
    df = df.drop('location', axis = 1)
    df = df.join(dummies)
    
    num_cols, cat_cols = df.columns[:8], df.columns[8:]
    df = pd.concat([
        pd.DataFrame(
            data=scaler.transform(df[num_cols]),
            columns=num_cols,
            index=df.index
        ),
        df[cat_cols]
        ],
        axis=1
    )
    
    return df

    
input_df = df.iloc[[0]]
input_df
output_df = preprocess_inference(input_df)

In [6]:
input_df

,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Avgift,Bostadstyp,Driftskostnad,Våning,Byggår,BRF,adress,area_and_n_rooms,location,url,Energiklass
0,5 950 000 kr,4 690 000 kr,+1 260 000 kr (+26.9 %),2021-05-05,108 200 kr/m²,13,3 178 kr/mån,Lägenhet,400 kr/mån,5 tr,1904.0,BRF Bergskyrkan,Bergs­gatan 59,"55 m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4201785,NaN


In [7]:
output_df

,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
0,-0.150847,0.387663,-0.440631,1.041806,-0.811701,-0.089688,-0.254217,-0.2614,1,0,0,0,0,0


In [8]:
model_loaded = Regressor().load_from_checkpoint('../model_checkpoints/epoch=78-avg_rmse_val=7.63.ckpt')
model_loaded.eval()

with torch.no_grad():
    pred = model_loaded(torch.tensor(output_df.values).float())

print(pred.numpy())

[[11.688712]]
